In [1]:
from Brokers.Alpaca.Bots.Alpaca_keyes import API_KEY, SECRET_KEY
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.stock import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest

In [2]:
api_key = API_KEY
secret_key = SECRET_KEY

symbol = "TSLA"

stock_historical_data_client = StockHistoricalDataClient(api_key, secret_key)

In [3]:
now = datetime.now(ZoneInfo("America/New_York"))

req = StockBarsRequest(
    symbol_or_symbols = [symbol],
    timeframe=TimeFrame(amount = 15, unit = TimeFrameUnit.Minute), # specify timeframe
    start = now - timedelta(days = 50),                          # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                        # specify end datetime, default=now
    #limit = 2,                                               # specify limit
)

In [4]:
# timestamp is in UTC time
stock_historical_data_client.get_stock_bars(req).df

open    high     low   close   volume  \
symbol timestamp                                                              
TSLA   2024-11-05 09:30:00+00:00  248.3800  249.25  248.25  248.77  57263.0   
       2024-11-05 09:45:00+00:00  248.8400  248.85  248.11  248.43  37898.0   
       2024-11-05 10:00:00+00:00  248.3900  248.44  247.61  247.76  41178.0   
       2024-11-05 10:15:00+00:00  247.6900  248.42  247.45  248.42  35573.0   
       2024-11-05 10:30:00+00:00  248.3900  248.47  247.81  247.83  43338.0   
...                                    ...     ...     ...     ...      ...   
       2024-12-24 20:45:00+00:00  464.5160  464.94  464.43  464.91  61705.0   
       2024-12-24 21:00:00+00:00  464.9237  465.13  464.75  464.90  47355.0   
       2024-12-24 21:15:00+00:00  464.9800  465.01  464.60  465.01  28342.0   
       2024-12-24 21:30:00+00:00  465.0451  465.22  464.80  464.95  33077.0   
       2024-12-24 21:45:00+00:00  464.9600  465.37  464.96  465.21  74307.0   

                                  trade_count        vwap  
symbol timestamp                                           
TSLA   2024-11-05 09:30:00+00:00       1607.0  248.838688  
       2024-11-05 09:45:00+00:00       1247.0  248.396762  
       2024-11-05 10:00:00+00:00       1424.0  247.963601  
       2024-11-05 10:15:00+00:00        990.0  247.881659  
       2024-11-05 10:30:00+00:00       1023.0  248.252622  
...                                       ...         ...  
       2024-12-24 20:45:00+00:00       1910.0  464.734573  
       2024-12-24 21:00:00+00:00       1766.0  464.928598  
       2024-12-24 21:15:00+00:00       1355.0  464.846876  
       2024-12-24 21:30:00+00:00       1501.0  465.029377  
       2024-12-24 21:45:00+00:00       2087.0  465.141942  

[2210 rows x 7 columns]

I have 15 min to calculate the missing candle, I need to write a function which converts live datasream into a 15 min candle

In [5]:
import nest_asyncio
nest_asyncio.apply()

async def stock_data_stream_handler(data):
    print(data)

In [6]:
from alpaca.data.live.stock import StockDataStream

stock_data_stream_client = StockDataStream(api_key, secret_key)

symbols = [symbol]

stock_data_stream_client.subscribe_quotes(stock_data_stream_handler, *symbols)
stock_data_stream_client.subscribe_trades(stock_data_stream_handler, *symbols)

In [ ]:
import asyncio
from alpaca.data.live.stock import StockDataStream

stock_data_stream_client = StockDataStream(api_key, secret_key)

async def stock_data_stream_handler(data):
    print(data)

symbols = [symbol]

# Subscribe to quotes and trades
stock_data_stream_client.subscribe_quotes(stock_data_stream_handler, *symbols)
stock_data_stream_client.subscribe_trades(stock_data_stream_handler, *symbols)

async def run_stream():
    await stock_data_stream_client.run()

async def main():
    # Create a task for running the stream
    stream_task = asyncio.create_task(run_stream())

    # Let the stream run for a specific duration (e.g., 60 seconds)
    await asyncio.sleep(10)

    # Stop the stream client
    await stock_data_stream_client.close() # do I need close() or stop()?

    # Wait for the stream task to complete
    await stream_task

# Run the main function
asyncio.run(main())
